# Contexte

Ce notebook est la deuxième partie d'un projet ou nous devons fournir des analyses concernant la répartition de l'alimentation, la sous-nutrition mais également en comprendre les causes.

# Sommaire

* [Préparation du dataset](#chapter3)
    
    
* [Analyse](#chapter4)
    * [Manioc](#section_4_1)
    * [Aide alimentaire](#section_4_2)
    * [Répartition Humain/Animale](#section_4_3)

# Préparation du dataset
<a class="anchor" id="chapter3"></a>

In [1]:
#import des librairies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as mno

import plotly.express as px

pd.set_option('display.max_columns', None)

In [2]:
#import des données
aide = pd.read_csv("data/aide_alimentaire.csv", delimiter=",")
dispo = pd.read_csv("data/dispo_alimentaire.csv", delimiter=",")
pop = pd.read_csv("data/population.csv", delimiter=",")
inséc = pd.read_csv("data/sous_nutrition.csv", delimiter=",")

In [3]:
#changement des noms de colonnes dans les datasets
aide1 = aide.rename(columns={"Pays bénéficiaire": "Zone","Valeur": "prod_données_tonne","Produit":"prod_données"})
pop1 = pop.rename(columns={"Valeur": "population"})
inséc1 = inséc.rename(columns={"Valeur": "mean_und-alim_million"})

#remplacement des tranches d'années
inséc1 = inséc1.replace(['2012-2014', '2013-2015', '2014-2016', '2015-2017','2016-2018','2017-2019'],
                        ['2013', '2014', '2015', '2016','2017','2018'])

#changement du type
inséc1['Année'] = inséc1['Année'].astype('int64')

In [4]:
#jointure de toute les datasets
df = pd.merge(pd.merge(pd.merge(aide1,pop1,on=['Zone',"Année"],how='outer'),inséc1,on=['Zone',"Année"],how='outer'),dispo,on='Zone')

In [15]:
#imputationdes valeur <0.1 par 0
df['mean_und-alim_million'] = df['mean_und-alim_million'].replace({"<0.1": 0})

In [16]:
#changement du type de la colonne
df['mean_und-alim_million'] = df['mean_und-alim_million'].astype('float')

In [17]:
#df

In [18]:
#filtrage pour l'année 2017
df_2017 = df[df['Année'] == 2017]

# Analyses

## Manioc

In [43]:
#selection du manioc et de la thaïlande
thai = df_2017[df_2017['Produit'] == 'Manioc']
thai = thai[thai['Zone'] == 'Thaïlande']

In [40]:
#passage de la population de millier en million
thai['population'] = thai['population']*1e-3

In [41]:
thai

,Zone,Année,prod_données,prod_données_tonne,population,mean_und-alim_million,Produit,Origine,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock
183588,Thaïlande,2017,NaN,NaN,69.20981,6.2,Manioc,vegetale,1800.0,2081.0,40.0,13.0,0.05,0.14,6264.0,25214.0,1250.0,871.0,1511.0,30228.0,NaN,0.0,0.0


In [25]:
#calcul de la proportion de sous_nutrition
thai['proportion_under'] = thai['mean_und-alim_million']/thai['population']*100

In [34]:
#calcul du taux d'export par la production
thai['proportion_export'] = thai['Exportations - Quantité']/thai['Production']*100

In [37]:
thai = thai[['population','mean_und-alim_million','Produit','Exportations - Quantité','proportion_under','proportion_export']]

In [38]:
thai

,population,mean_und-alim_million,Produit,Exportations - Quantité,proportion_under,proportion_export
183588,69.20981,6.2,Manioc,25214.0,8.958268,83.41273


## Aide alimentaire

In [50]:
aide = df.groupby(['Zone']).sum()

In [52]:
aide = aide[['prod_données_tonne']]

In [54]:
aide.sort_values(by=['prod_données_tonne'], ascending= False).head(10)

,prod_données_tonne
Zone,
Éthiopie,127079048.0
Yémen,112203012.0
Soudan,56261856.0
Kenya,53072256.0
Bangladesh,32729672.0
Niger,25147304.0
Pakistan,21720768.0
Tchad,19561518.0
République-Unie de Tanzanie,16514112.0


## Répartition Humain/Animale

In [97]:
repart = df_2017[df_2017['Produit'].isin(['Blé','Céréales, Autres','Soja','Seigle','Sorgho','Riz (Eq Blanchi)','Orge','Maïs','Millet'])]

In [98]:
repart = repart[['Zone','Aliments pour animaux','Disponibilité alimentaire en quantité (kg/personne/an)','Disponibilité intérieure']]

In [99]:
#création d'une variable qui stockera la population correcte
var_pop = df_2017[['Zone','population']]
var_pop = var_pop.drop_duplicates()

In [100]:
repartition = repart.merge(var_pop, on='Zone')

In [101]:
repartition = repartition.groupby(['Zone']).sum()

In [102]:
repartition['alim_pop'] = repartition['population'] * repartition['Disponibilité intérieure']

In [103]:
#repartition['alim_pop'] = repartition['alim_pop']*1e-6

In [106]:
repartition['proportion_animale'] = repartition['Aliments pour animaux']/repartition['Disponibilité intérieure']*100
repartition['proportion_humain'] = repartition['Disponibilité alimentaire en quantité (kg/personne/an)']/repartition['Disponibilité intérieure']*100

In [107]:
repartition

,Aliments pour animaux,Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité intérieure,population,alim_pop,proportion_animale,proportion_humain
Zone,,,,,,,
Afghanistan,560.0,179.87,7303.0,217776.678,1.590423e+09,7.668082,2.462960
Afrique du Sud,4824.0,180.54,16176.0,513087.804,8.299708e+09,29.821958,1.116098
Albanie,386.0,149.14,1117.0,25957.521,2.899455e+07,34.556849,13.351835
Algérie,4230.0,217.66,14891.0,372502.701,5.546938e+09,28.406420,1.461688
Allemagne,26043.0,110.45,44786.0,743925.681,3.331746e+10,58.149868,0.246617
...,...,...,...,...,...,...,...
Émirats arabes unis,1055.0,158.76,3304.0,85384.827,2.821115e+08,31.930993,4.805085
Équateur,847.0,92.61,3447.0,151068.249,5.207323e+08,24.572092,2.686684
États-Unis d'Amérique,140906.0,101.72,392881.0,2925762.804,1.149477e+12,35.864804,0.025891


In [108]:
df

,Zone,Année,prod_données,prod_données_tonne,population,mean_und-alim_million,Produit,Origine,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock
0,Afghanistan,2013,Autres non-céréales,682.0,32269.589,8.6,Abats Comestible,animale,NaN,NaN,5.0,1.72,0.20,0.77,53.0,NaN,NaN,53.0,NaN,53.0,NaN,NaN,NaN
1,Afghanistan,2013,Autres non-céréales,682.0,32269.589,8.6,"Agrumes, Autres",vegetale,NaN,NaN,1.0,1.29,0.01,0.02,41.0,2.0,40.0,39.0,2.0,3.0,NaN,NaN,NaN
2,Afghanistan,2013,Autres non-céréales,682.0,32269.589,8.6,Aliments pour enfants,vegetale,NaN,NaN,1.0,0.06,0.01,0.03,2.0,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,2013,Autres non-céréales,682.0,32269.589,8.6,Ananas,vegetale,NaN,NaN,0.0,0.00,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,2013,Autres non-céréales,682.0,32269.589,8.6,Bananes,vegetale,NaN,NaN,4.0,2.70,0.02,0.05,82.0,NaN,82.0,82.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187881,Viet Nam,2018,NaN,NaN,95545.962,6.1,Viande de Suides,animale,NaN,NaN,344.0,35.00,33.02,10.61,3209.0,10.0,1.0,3209.0,NaN,3218.0,NaN,NaN,NaN
187882,Viet Nam,2018,NaN,NaN,95545.962,6.1,Viande de Volailles,animale,NaN,NaN,46.0,12.36,3.24,4.04,1133.0,0.0,500.0,1133.0,NaN,633.0,NaN,NaN,NaN
187883,Viet Nam,2018,NaN,NaN,95545.962,6.1,"Viande, Autre",animale,NaN,NaN,1.0,0.29,0.05,0.13,27.0,0.0,NaN,27.0,NaN,27.0,NaN,NaN,NaN
187884,Viet Nam,2018,NaN,NaN,95545.962,6.1,Vin,vegetale,NaN,NaN,0.0,0.13,NaN,NaN,12.0,NaN,12.0,12.0,NaN,NaN,NaN,NaN,NaN
